In [ ]:
#Analysis and Visualisation

import pandas as pd
# Auxiliary classes:
from dataframeinfo import DataFrameInfo
from datatransform import DataTransform
from plotter import Plotter

In [ ]:
# Importing the transformed data into dataframes
df = pd.read_csv('loan_payments_versions/loan_payments_post_null_imputation.csv', index_col='id') # Pre normalised dataframe.
normalised_df = pd.read_csv('loan_payments_versions/loan_payments_transformed.csv', index_col='id') # Normalised datafram

In [ ]:
# Instantiating classes:
transform = DataTransform()
query = DataFrameInfo()
visual = Plotter()

In [ ]:
# change string to date formats.
transform.convert_string_to_date(df, 'last_payment_date')
transform.convert_string_to_date(df, 'issue_date')

transform.convert_string_to_date(normalised_df, 'last_payment_date')
transform.convert_string_to_date(normalised_df, 'issue_date')

In [ ]:
# Calculate the overall percentage of recovery against total and investor funding.
total_recovery_percentage = query.calculate_column_percentage(df, 'total_payment', 'funded_amount')
total_recovery_percentage_inv = query.calculate_column_percentage(df, 'total_payment', 'funded_amount_inv')

# Visualize the results using a bar plot.
categories = ['Total Funding', 'Investor Funding']
variables = [total_recovery_percentage, total_recovery_percentage_inv]
visual.bar_chart(independant_categories=categories, dependant_variables=variables, title='Percentage of Loans Recovered Against Funding', y_label='Recovery Percentage')


In [ ]:
# Visualise the results using a pie chart.
visual.pie_chart(labels=categories, sizes=variables, title='Percentage of Loans Recovered Against Funding')

In [ ]:
collection_projections = query.calculate_total_collections_over_period(df, period=6) # Obtain the collection amount, loan amount and outstanding loan for the 6 month period in a dictionary.

total_collections = collection_projections['total_collections'] # Extract total collection amount from dictionary.
total_loan = collection_projections['total_loan'] # Extract total loan amount from dictionary.
total_loan_outstanding = collection_projections['total_loan_outstanding'] # Extract total loan amount outstanding from dictionary.

collections_percent = query.calculate_percentage(total_collections, total_loan) # calculate collections over total loan for 6 months as a percentage. 
print(f'The collections over the next 6 months are equivelant to {round(total_collections,2)}\nThis is {round(collections_

In [ ]:
# Setting variables for pie chart.
sizes_1 = [total_collections, total_loan]
sizes_2 = [total_collections, total_loan_outstanding]
# Setting labels for pie chart.
labels_1 = ['Collections in 6 months', 'Total loan']
labels_2 = ['Collections in 6 months', 'Total loan outstanding']
# Generating two pie charts to compare collections by total loan and outstanding loan.
visual.two_pie_charts(sizes_1=sizes_1, labels_1=labels_1, sizes_2=sizes_2, labels_2=labels_2, plot_title='Collections over 6 months'

In [ ]:
projected_percentages_dictionary = query.monthly_collection_percentage_projections(df, 6)

# Setting up bar chart plot parameters.
categories = [1,2,3,4,5,6] # identifying categories in terms of months of projection.
variables_1 = projected_percentages_dictionary['total_loan_percent'] # defining percentage variables for first bar chart by extracting first dictionary list.
variables_2 = projected_percentages_dictionary['outstanding_loan_percent'] # defining percentage variables for second bar chart by extracting second dictionary list.
title_1 = 'collections as percentage of total loan'
title_2 = 'collections as percentage of outstanding loan'
y_label = 'percentage'
x_label ='Month'

# Generating plot:
visual.two_bar_charts(independant_categories_1=categories, dependant_variables_1=variables_1, independant_categories_2=categories, dependant_variables_2=variables_2,
                     plot_title='Collections compared to loan', title_1=title_1, title_2=title_2, y_label_1=y_label, x_label_1=x_label)

In [ ]:
number_of_charge_offs = query.count_value_in_column(df, 'loan_status', 'Charged Off') # Total number of loans that were 'Charged Off'.
charge_off_percentage = query.calculate_percentage(number_of_charge_offs, len(df)) # Percentage of total loans that were 'Charged Off'.

# Generate plot comparing number of charged off to non charged off loans.
visual.pie_chart(labels=['Charged Off', 'Not Charged Off'], sizes=[charge_off_percentage, 100 - charge_off_percentage],)

In [ ]:
funded_amounts_charged_off = df[df['loan_status']=='Charged Off']['funded_amount'].sum() # Total amount paid out in 'Charged Off' loans.
total_payment_charged_off = df[df['loan_status']=='Charged Off']['total_payment'].sum() # Total amount paid towards loan by customers before they were 'Charged Off'.
percent_paid_charged_off = query.calculate_percentage(total_payment_charged_off, funded_amounts_charged_off)

print(f"The total value of all charged off loans is {round(funded_amounts_charged_off,2)}.\nA total of {round(total_payment_charged_off,2)} was paid before these loans were charged off.")
print(f"This means the company recieved {round(percent_paid_charged_off,1)} % of funds for 'Charged Off' loans.")

# Generating pie chart to visualise data
visual.pie_chart(labels=['Amount Paid', 'Amount Not Paid'], sizes=[percent_paid_charged_off, 100-percent_paid_charged_off])

In [ ]:
charged_off_df = df[df['loan_status']=='Charged Off'] # Filtering out non 'Charged Off' loans.
total_loan_amount = charged_off_df['funded_amount'] + (charged_off_df['funded_amount'] * (charged_off_df['int_rate']) * 0.01) # Total amount owed by customer, accounting for interest.
charged_off_df['projected_loss'] = total_loan_amount - charged_off_df['total_payment'] # Calculating the projected loss for each 'Charged Off' loan.
projected_loss = charged_off_df['projected_loss'].sum() # Calculating total loss of revenue for 'Charged Off' loans.

# Generating bar chart to visualise loss.
categories = ['Total Expected Revenue', 'Total Actual Revenue', 'Total Loans Value', 'Charged Off Loan Paid Back Total', 'Projected Loss of Revenue']
total_expected_revenue = (df['loan_amount'] * (1 + (df['int_rate'] * 0.01))).sum() # Convert interest rate to decimal and factor that into loan_amount
total_actual_revenue = df['total_payment'].sum()
loans_total = df['funded_amount'].sum()
data = [total_expected_revenue/1000000000, total_actual_revenue/1000000000, loans_total/1000000000, total_payment_charged_off/1000000000, projected_loss/1000000000] # Categorical variables divided by 1,000,000,000.

visual.bar_chart(independant_categories=categories, dependant_variables=data, title="Projected Loss in Revenue from 'Charged Off Loan Paid Back Total")

In [ ]:
revenue_lost = query.revenue_lost_by_month(charged_off_df) # Calculating cumulative revenue lost for each month in a list.
revenue_lost = [revenue/1000000 for revenue in revenue_lost] # Scaling the revenue values down.
# Visualising cumulative projected revenue lost for each month instalments are paid:
visual.bar_chart(independant_categories=list(range(1, len(revenue_lost)+1)), dependant_variables=revenue_lost, title=revenue_lost)

In [ ]:
risky_status = ['Late (31-120 days)', 'In Grace Period', 'Late (16-30 days)'] # All the risky loan status.
risky_df = df[df['loan_status'].isin(risky_status)] # A dataframe that contains only loans deemed as risky status.
current_df = df[df['loan_status'].isin(['Current','Late (31-120 days)','In Grace Period', 'Late (16-30 days)'])] # A mask for all current loans in dataframe.

risky_customers_sum = len(risky_df) # Returns the number of risky loans.

percentage_of_risky_loans = query.calculate_percentage(risky_customers_sum, len(df)) # Percentage of risky loans out of all loans.
percentage_of_current_risky_loans = query.calculate_percentage(risky_customers_sum, len(current_df)) # Percentage of risky loans out of all current loans.

print(f"There are {risky_customers_sum} 'Risky' loans.")

# Generating pie chart visualisation of risky loan percentage out of 1) All loans 2) Current loans:
visual.two_pie_charts(labels_1=['Risky Loans', 'Non Risky Loans'], labels_2=['Risky Loans', 'Non Risky Loans'], sizes_1=[percentage_of_risky_loans, 100-percentage_of_risky_loans],
                    sizes_2=[percentage_of_current_risky_loans, 100-percentage_of_current_risky_loans], title_1='All Loans') 

In [ ]:
risky_projected_revenue_lost = query.revenue_lost_by_month(risky_df) # Calculating cumulative revenue lost for each month in a list.
risky_total_revenue_lost = risky_projected_revenue_lost[-1] # Total revenue lost from risky loans.
risky_total_expected_revenue = query.calculate_total_expected_revenue(risky_df) # Expected total revenue from all risky loans, if paid fully.
current_total_expected_revenue = current_df['total_payment'].sum() #  Total expected revenue from currently active loans.

print(f"\nIf 'Risky' loans were 'Charged Off' the company would expect to lose {round(risky_total_revenue_lost,2)} amount of revenue.")

# Setting up parameters for visualisation:
label = ['Total Projected Revenue Gained', 'Total Projected Revenue Loss']
variables_1 = [risky_total_expected_revenue-risky_total_revenue_lost, risky_total_revenue_lost]
variables_2 = [current_total_expected_revenue-risky_total_revenue_lost, risky_total_revenue_lost]
title = 'Potential Revenue Lost from Risky Loans'
title_1 = 'Projected Revenue Lost from Risky Loans'
title_2 = "Projected Revenue Lost from Total Current Loans' Expected Revenue"

# Generating visualisation:
visual.two_pie_charts(labels_1=label, sizes_1=variables_1, labels_2=label, sizes_2=variables_2, title_1=title_1, title_2=title_2)

In [ ]:
default_df = df[df['loan_status']=='Default'] # A dataframe that contains only loans that were defaulted.
default_projected_revenue_lost = query.revenue_lost_by_month(default_df) # Calculating cumulative revenue lost for each month in a list.
default_total_revenue_lost = default_projected_revenue_lost[-1] # Total revenue lost from defaulted loans

# Setting up parameters for visualisation:
variables_3 = [total_expected_revenue-default_total_revenue_lost, default_total_revenue_lost]
title_3 = 'Projected Revenue Lost from Default Loans'

# Generating Visualisation:
visual.pie_chart(labels=label, sizes=variables_3, title=title_3)

In [ ]:
visual.discrete_value_risk_comparison(df, 'grade')

In [ ]:
visual.discrete_value_risk_comparison(df, 'term')

In [ ]:
visual.discrete_value_risk_comparison(df, 'employment_length')

In [ ]:
visual.discrete_value_risk_comparison(df, 'home_ownership')

In [ ]:
visual.discrete_value_risk_comparison(df, 'purpose')

In [ ]:
visual.continuous_value_risk_comparison(df, 'annual_inc')

In [ ]:
visual.continuous_value_risk_comparison(df, 'int_rate')

In [ ]:
visual.continuous_value_risk_comparison(df, 'loan_amount')

In [ ]:
visual.continuous_value_risk_comparison(df, 'instalment')

In [ ]:
visual.continuous_value_risk_comparison(df, 'dti')